<a href="https://colab.research.google.com/github/Huwalli/PLta/blob/main/%E7%B6%B2%E8%B7%AF%E8%B3%87%E6%96%99%E8%87%AA%E5%8B%95%E7%88%AC%E5%8F%96%E8%88%87%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ 1. 從網路上抓取新聞（Web Scraping）

以 ptt 為例，使用 requests + BeautifulSoup 抓取新聞標題與內容。

✅ 2. 分析新聞內容（NLP 分析）

用 SnowNLP（中文）進行情緒分析。

✅ 3. 視覺化分析結果

用 pandas 做表格處理，plotly 視覺化結果。

In [1]:
!pip install snownlp plotly beautifulsoup4 # 安裝中文 NLP、互動視覺化與 HTML 解析套件

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=5d4796a3b4a4d3b65056ddd307acd19ff196a2afc54db3704145ae89ed26b9cd
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [2]:
# 匯入 requests 模組，用來對網頁伺服器發送請求 (Request) 並取得回應 (Response)
import requests

# 從 bs4 套件中匯入 BeautifulSoup，用來解析 HTML 文件，方便從網頁中找出特定資料
from bs4 import BeautifulSoup

# 匯入 SnowNLP 套件，用來進行中文的自然語言處理（像是情感分析、斷詞）
from snownlp import SnowNLP

# 匯入 pandas，用來整理資料成表格（DataFrame），方便後續分析與儲存
import pandas as pd

# 匯入 plotly.express，這是畫互動圖表的套件，可以用來視覺化分析結果
import plotly.express as px

# 設定瀏覽器標頭（headers），模擬成一般使用者開啟網頁，避免被網站封鎖
HEADERS = {
    'User-Agent': 'Mozilla/5.0'  # 模擬成常見的瀏覽器（像是 Chrome）
}

# 設定 cookies，這是針對某些版面（如 PTT Gossiping）會要求確認年齡，這裡模擬已經按下「我已滿18歲」
COOKIES = {'over18': '1'}


In [3]:
# 定義函式 get_ptt_articles，預設抓取 PTT Stock 板的前 2 頁文章
def get_ptt_articles(board='Stock', max_pages=2):
    # 設定第一頁的網址
    base_url = f'https://www.ptt.cc/bbs/{board}/index.html'

    # 用來儲存所有文章資料的清單
    articles = []

    # 重複爬 max_pages 頁
    for _ in range(max_pages):
        # 發送 GET 請求取得該頁 HTML 原始碼，附上 headers 和 cookies 模擬使用者
        res = requests.get(base_url, headers=HEADERS, cookies=COOKIES)
        # 用 BeautifulSoup 解析 HTML
        soup = BeautifulSoup(res.text, 'html.parser')
        # 找出所有文章的區塊（div.r-ent 是每篇文章的外層容器）
        entries = soup.select('div.r-ent')

        # 逐篇處理文章
        for entry in entries:
            # 找出文章標題的 <a> 標籤
            title_tag = entry.select_one('div.title > a')
            if title_tag:
                # 取得文章標題文字，去除前後空白
                title = title_tag.text.strip()
                # 製作完整的文章網址
                link = 'https://www.ptt.cc' + title_tag['href']

                try:
                    # 再次發送請求進入文章頁面
                    article_res = requests.get(link, headers=HEADERS, cookies=COOKIES)
                    article_soup = BeautifulSoup(article_res.text, 'html.parser')
                    # 抓取主內容區（#main-content）並切除簽名檔（用 -- 分隔）
                    content = article_soup.select_one('#main-content').text.split('--')[0]
                    # 去除以「※」開頭的系統資訊
                    content = '\n'.join(line for line in content.split('\n') if not line.startswith('※'))
                    # 用 SnowNLP 計算情緒分數（0 為負面，1 為正面）
                    sentiment = SnowNLP(content).sentiments
                except Exception as e:
                    # 若發生錯誤（例如請求失敗），則將內容設為空，情緒分數為 None
                    content = ""
                    sentiment = None

                # 將資料加入結果清單
                articles.append({
                    '標題': title,
                    '連結': link,
                    '情緒分數': sentiment
                })

        # 嘗試找出「上一頁」的連結（第二個按鈕通常是上一頁）
        paging = soup.select_one('div.btn-group-paging > a.btn.wide:nth-child(2)')
        if paging:
            # 更新 base_url 為上一頁的網址，準備下一輪爬取
            base_url = 'https://www.ptt.cc' + paging['href']
        else:
            # 如果找不到下一頁，跳出迴圈
            break

    # 最後將所有資料轉換為 pandas 資料表並回傳
    return pd.DataFrame(articles)


In [4]:
# 抓 PTT Stock 看板 2 頁
df = get_ptt_articles('Stock', max_pages=2)
df = df.dropna(subset=['情緒分數'])

In [5]:
df

,標題,連結,情緒分數
0,Re: [新聞] 台積電19次秒填息沒了！陸行之揭1原因很,https://www.ptt.cc/bbs/Stock/M.1749722829.A.BC...,8.881784e-16
1,[情報] 2323 中環 取得陽明 均價72.32元,https://www.ptt.cc/bbs/Stock/M.1749724782.A.B9...,0.000000e+00
2,[新聞] 美多座晶片廠難動工狂燒錢、居民喊別蓋我家後院,https://www.ptt.cc/bbs/Stock/M.1749725296.A.1C...,0.000000e+00
3,[請益] 匯率33塊多去歐印美債是什麼人,https://www.ptt.cc/bbs/Stock/M.1749728527.A.35...,8.218492e-01
4,Re: [請益] 匯率33塊多去歐印美債是什麼人,https://www.ptt.cc/bbs/Stock/M.1749729124.A.4B...,6.983497e-01
5,[新聞] 避免倒閉潮引發房市崩盤危機 達麗董座,https://www.ptt.cc/bbs/Stock/M.1749729165.A.67...,0.000000e+00
6,[請益] 沒上車的會後悔一輩子嗎?,https://www.ptt.cc/bbs/Stock/M.1749731214.A.EB...,7.002998e-11
7,Re: [標的] 00915 進場在22塊，是撿到還是接刀？,https://www.ptt.cc/bbs/Stock/M.1749731693.A.66...,0.000000e+00
8,[請益] 一次投入大筆資金長期投資的最佳選擇是？,https://www.ptt.cc/bbs/Stock/M.1749734131.A.24...,3.518269e-07
9,[新聞] 新台幣升勢再起！匯銀主管示警：該避的險,https://www.ptt.cc/bbs/Stock/M.1749736017.A.23...,0.000000e+00


In [6]:
# 畫圖
fig = px.bar(
    df.sort_values('情緒分數'),
    x='情緒分數',
    y='標題',
    orientation='h',
    title='PTT 文章情緒分析',
    height=600
)
fig.update_layout(yaxis=dict(automargin=True))
fig.show()